In [1]:
import nltk
nltk.download
nltk.download('punkt')
from nltk.corpus import stopwords

stopwords_en = stopwords.words ("english")

def preprocessing (raw):
    word_list = nltk.word_tokenize (raw)
    text = [w.lower() for w in word_list if w not in stopwords_en]
    return text

f1 = open('summary1.txt', 'r')
text1 = preprocessing (f1.read())

f2 = open ('summary2.txt', 'r')
text2 = preprocessing (f2.read())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\porus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from nltk.probability import FreqDist

word_set = set(text1).union(set(text2))

freqd_text1 = FreqDist(text1)
text1_count_dict = dict.fromkeys(word_set,0)
for word in text1:
    text1_count_dict[word] = freqd_text1[word]

freqd_text2 = FreqDist(text2)
text2_count_dict = dict.fromkeys(word_set,0)
for word in text2:
    text2_count_dict[word]=freqd_text2[word]
print(text1_count_dict)
print(text2_count_dict)

{'agreed': 1, 'town': 2, 'could': 1, 'affection': 1, 'helpless': 2, 'dead': 1, 'feet': 2, 'become': 2, 'flock': 0, 'experienced': 0, 'it': 2, 'remove': 2, 'asks': 0, 'weep': 1, 'went': 1, 'breaking': 1, 'bird': 15, 'melted': 1, 'melt': 1, 'plight': 2, 'embedded': 1, 'by': 1, 'among': 1, 'blind': 1, 'explains': 0, 'asked': 3, 'way': 2, 'came': 1, 'suffering': 0, 'crying': 0, 'convinced': 1, 'playwright': 1, 'inform': 1, 'grey': 1, 'death': 2, 'goodbye': 1, 'help': 1, 'egypt': 1, 'became': 2, 'this': 1, 'stone': 4, 'place': 1, 'him': 0, 'things': 1, 'gemstones': 0, 'rests': 0, 'breaks': 0, 'discarded': 0, 'bring': 0, 'called': 1, 'he': 1, 'becoming': 1, 'overwhelmed': 1, 'stay': 1, 'unable': 1, 'merry': 1, 'took': 3, 'nothing': 0, 'handle': 1, 'covering': 1, 'tall': 1, 'lifetime': 0, 'never': 0, 'leave': 2, 'swallow': 4, 'devoid': 0, 'decides': 0, 'pained': 0, 'young': 1, 'developed': 1, 'furnace': 1, 'gone': 0, 'want': 1, 'back': 1, 'deliver': 1, 'lead': 1, 'now': 0, 'dies': 0, 'surviva

In [3]:
freqd_text1 = FreqDist (text1)
text1_length = len(text1)
text1_tf_dict = dict.fromkeys(word_set, 0)
for word in text1:
    text1_tf_dict[word] = freqd_text1[word]/text1_length

freqd_text2 = FreqDist(text2)
text2_length = len(text2)
text2_tf_dict = dict.fromkeys(word_set, 0)
for word in text2:
    text2_tf_dict[word] = freqd_text2[word]/text2_length

In [4]:
text12_idf_dict = dict.fromkeys (word_set,0)
text12_length = 2 # 2 documents
for word in text12_idf_dict.keys () :
    if word in text1:

        text12_idf_dict [word] += 1
    
    if word in text2:
        text12_idf_dict [word] += 1

import math
for word, val in text12_idf_dict.items():
    text12_idf_dict[word] = 1 + math.log(text12_length/ (float(val) ) )

In [5]:
# TF-IDF Calculations
text1_tfidf_dict = dict.fromkeys (word_set, 0)
for word in text1:
    text1_tfidf_dict[word] = (text1_tf_dict[word]) * (text12_idf_dict[word])

text2_tfidf_dict = dict.fromkeys(word_set, 0)
for word in text2:
    text2_tfidf_dict[word] = (text2_tf_dict[word]) * (text12_idf_dict[word])

In [6]:
import gensim.models.doc2vec
from gensim.models.doc2vec import TaggedDocument

taggeddocs = []
doc1 = TaggedDocument(words=text1, tags=[u'NEWS_1'])
taggeddocs.append(doc1)
doc2 = TaggedDocument(words=text2, tags=[u'NEWS_2'])
taggeddocs.append(doc2)

# build the model
model = gensim.models.Doc2Vec(taggeddocs, dm=0, alpha=0.025, size=20, min_alpha=0.025, min_count=0)

# training
for epoch in range (80):
    if epoch % 20 == 0:
        print ('Now training epoch %s' % epoch)
    
    model.train(taggeddocs,total_examples=model.corpus_count,epochs=model.epochs)
    model.alpha -= 0.002 # decrease the learning rate
    model.min_alpha = model.alpha # fix the learning rate, no decay

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


Now training epoch 0
Now training epoch 20
Now training epoch 40
Now training epoch 60


In [7]:
# Compute Cosine distance
v1 = list(text1_tfidf_dict.values())
v2 = list(text2_tfidf_dict.values ())
similarity = 1 - nltk.cluster.cosine_distance(v1,v2)
print ("Similarity Index:{:4.2f} %".format(similarity*100))

Similarity Index:60.84 %


In [8]:
similarity = model.n_similarity (text1, text2)
print ("Similarity Index: {:4.2f} %".format(similarity*100))

Similarity Index: 82.85 %


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.
